In [23]:
from ultralytics import YOLO
from pathlib import Path
import torch

DATA_YAML = r"C:\Final project 2\backend\datasets\data.yaml"
RUNS_DIR  = r"C:\Final project 2\backend\runs"

print("Torch:", torch.__version__, "| CUDA:", torch.version.cuda, "| GPU available:", torch.cuda.is_available())

device_arg = 0 if torch.cuda.is_available() else "cpu"

model = YOLO("yolov8s.pt")  # start small; switch to yolov8s/8m later

model.train(
    data=DATA_YAML,
    epochs=50,
    imgsz=640,
    batch=16,
    project=RUNS_DIR,
    name="yolo8s-fracture",
    device=device_arg,     # <-- auto-picks 0 or cpu
    deterministic=True
)


Torch: 2.8.0+cu129 | CUDA: 12.9 | GPU available: True
Ultralytics 8.3.225  Python-3.10.18 torch-2.8.0+cu129 CUDA:0 (NVIDIA GeForce RTX 5070 Laptop GPU, 8151MiB)
engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=C:\Final project 2\backend\datasets\data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8s.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=yolo8s-fracture, nbs=64, nms=False, opset=None

ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x000001EB1703D870>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,   

In [2]:
import torch, sys
print("Exe:", sys.executable)
print("Torch:", torch.__version__, "CUDA:", torch.version.cuda, "Available:", torch.cuda.is_available())
print("Devices:", torch.cuda.device_count())
print(torch.cuda.get_device_name(0) if torch.cuda.is_available() else "CPU-only")


Exe: c:\Users\aadit\anaconda3\envs\fxdetector\python.exe
Torch: 2.8.0+cu129 CUDA: 12.9 Available: True
Devices: 1
NVIDIA GeForce RTX 5070 Laptop GPU


In [15]:
# --- 02_export_and_validate.ipynb ---
from ultralytics import YOLO
from pathlib import Path

# === Paths ===
BASE_DIR = Path(r"C:\Final project 2\backend")

# your new model path 👇
MODEL_PT = BASE_DIR / r"runs" / r"yolo8s-fracture" / r"weights" / r"best.pt"

# where to save exported models
EXPORT_DIR = BASE_DIR / "models" / "exported"
EXPORT_DIR.mkdir(parents=True, exist_ok=True)

# your dataset yaml (make sure this points to the right train/val/test)
DATA_YAML = BASE_DIR / "datasets" / "data.yaml"

# === Load model ===
model = YOLO(str(MODEL_PT))
print("✅ Model loaded from:", MODEL_PT)

# === Validate on your dataset ===
results = model.val(
    data=str(DATA_YAML),
    imgsz=640,
    conf=0.25,
    device=0,         # set to 0 for GPU, or 'cpu'
    plots=True,
    save_json=True
)

# --- Print summary ---
print("\n📊 Validation summary:")
print(f"Precision(B): {results.results_dict['metrics/precision(B)']:.3f}")
print(f"Recall(B):    {results.results_dict['metrics/recall(B)']:.3f}")
print(f"mAP50(B):     {results.results_dict['metrics/mAP50(B)']:.3f}")
print(f"mAP50–95(B):  {results.results_dict['metrics/mAP50-95(B)']:.3f}")
print(f"Fitness:      {results.results_dict['fitness']:.3f}")

# === Export to ONNX and TorchScript for deployment ===
print("\n🧩 Exporting model formats (ONNX + TorchScript)...")

# ONNX export
model.export(
    format="onnx",
    dynamic=True,
    opset=12,
    imgsz=640,
    half=False,
    optimize=True,
    project=str(EXPORT_DIR),
    name="yolo8s-fracture-onnx"
)

# TorchScript export
model.export(
    format="torchscript",
    imgsz=640,
    project=str(EXPORT_DIR),
    name="yolo8s-fracture-torchscript"
)

print("\n✅ Export complete. Files saved under:")
for f in EXPORT_DIR.glob("*.*"):
    print(" ", f.name)

# === Quick test on validation images ===
TEST_IMG_DIR = BASE_DIR / "datasets" / "valid" / "images"
preds = model.predict(
    source=str(TEST_IMG_DIR),
    imgsz=640,
    conf=0.10,
    save=True,
    device=0
)
print(f"\n✅ Saved predictions to: {preds[0].save_dir}")

# (optional, if you're in a notebook)
# from IPython.display import Image, display
# for p in Path(results.save_dir).glob("*.png"):
#     display(Image(filename=str(p)))


✅ Model loaded from: C:\Final project 2\backend\runs\yolo8s-fracture\weights\best.pt
Ultralytics 8.3.225  Python-3.10.18 torch-2.8.0+cu129 CUDA:0 (NVIDIA GeForce RTX 5070 Laptop GPU, 8151MiB)
Model summary (fused): 72 layers, 11,129,454 parameters, 0 gradients, 28.5 GFLOPs
val: Fast image access  (ping: 0.00.0 ms, read: 481.7137.8 MB/s, size: 24.5 KB)
val: Scanning C:\Final project 2\backend\datasets\valid\labels.cache... 128 images, 0 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 128/128 127.4Kit/s 0.0s
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 8/8 3.1it/s 2.6s0.2ss
                   all        128        157       0.92      0.806      0.913      0.496
            Comminuted         15         15          1      0.594      0.835      0.572
            Greenstick         10         10      0.846      0.551      0.734      0.301
               Healthy          7          7          1          1      0.995      0.748
        

In [22]:
import os
from collections import Counter

def count_labels(label_dir):
    counts = Counter()
    for file in os.listdir(label_dir):
        if file.endswith(".txt"):
            with open(os.path.join(label_dir, file)) as f:
                for line in f:
                    cls = int(line.split()[0])
                    counts[cls] += 1
    return counts

train_counts = count_labels(r"C:\Final project 2\backend\datasets\train\labels")
val_counts = count_labels(r"C:\Final project 2\backend\datasets\valid\labels")

print("Train label counts:", train_counts)
print("Validation label counts:", val_counts)


Train label counts: Counter({9: 709, 0: 698, 8: 680, 7: 650, 4: 650, 2: 650, 6: 650, 5: 650, 1: 650, 3: 650})
Validation label counts: Counter({8: 65, 4: 30, 0: 15, 9: 14, 1: 10, 2: 7, 5: 7, 7: 5, 6: 3, 3: 1})


In [10]:
from ultralytics import YOLO
MODEL_PATH = r"C:\Final project 2\backend\models\best.pt"  # <-- adjust if needed
model = YOLO(MODEL_PATH)
print(model.names)  # should be a dict {0:'elbow',1:'finger',...}


{0: 'Comminuted', 1: 'Greenstick', 2: 'Healthy', 3: 'Linear', 4: 'Oblique Displaced', 5: 'Oblique', 6: 'Segmental', 7: 'Spiral', 8: 'Transverse Displaced', 9: 'Transverse'}


In [15]:
import os

def find_unlabeled(img_dir, label_dir):
    imgs = [f[:-4] for f in os.listdir(img_dir) if f.lower().endswith(('.jpg', '.jpeg', '.png'))]
    labels = [f[:-4] for f in os.listdir(label_dir) if f.lower().endswith('.txt')]
    unlabeled = [f for f in imgs if f not in labels]
    return unlabeled

train_unlabeled = find_unlabeled(r"C:\Final project 2\backend\datasets\train\images",r"C:\Final project 2\backend\datasets\train\labels")
val_unlabeled = find_unlabeled(r"C:\Final project 2\backend\datasets\valid\images",r"C:\Final project 2\backend\datasets\valid\labels")

print("Unlabeled training images:", len(train_unlabeled))
print("Unlabeled validation images:", len(val_unlabeled))

if train_unlabeled:
    print("Examples:", train_unlabeled[:10])


Unlabeled training images: 0
Unlabeled validation images: 0


In [16]:
metrics = model.val()
print(f"mAP50: {metrics.box.map50:.3f}")
print(f"mAP50-95: {metrics.box.map:.3f}")
print(f"Precision: {metrics.box.p.mean():.3f}")
print(f"Recall: {metrics.box.r.mean():.3f}")

Ultralytics 8.3.225  Python-3.10.18 torch-2.8.0+cu129 CUDA:0 (NVIDIA GeForce RTX 5070 Laptop GPU, 8151MiB)
val: Fast image access  (ping: 0.00.0 ms, read: 552.9104.6 MB/s, size: 29.0 KB)
val: Scanning C:\Final project 2\backend\datasets\valid\labels.cache... 128 images, 0 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 128/128  0.0s
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 8/8 3.1it/s 2.6s0.2ss
                   all        128        157      0.914      0.812      0.906      0.472
            Comminuted         15         15          1      0.594      0.832      0.517
            Greenstick         10         10      0.846      0.551       0.59      0.225
               Healthy          7          7      0.941          1      0.995      0.731
                Linear          1          1      0.816          1      0.995      0.199
     Oblique Displaced         30         30      0.959      0.933       0.96      0.429
       

In [18]:
import os
from pathlib import Path
import cv2
import random
from collections import defaultdict

# ====== CONFIG ======
DATA_ROOT = Path(r"C:\Final project 2\backend\datasets\train")  # <- your path
IMG_DIR = DATA_ROOT / "images"
LBL_DIR = DATA_ROOT / "labels"

CLASS_NAMES = {
    0: 'Comminuted',
    1: 'Greenstick',
    2: 'Healthy',
    3: 'Linear',
    4: 'Oblique Displaced',
    5: 'Oblique',
    6: 'Segmental',
    7: 'Spiral',
    8: 'Transverse Displaced',
    9: 'Transverse'
}

TARGET_PER_CLASS = 650      # set the cap you want
AUG_TRIES_PER_IMAGE = 3     # how many aug versions to try per source
# =====================


def apply_clahe(img):
    lab = cv2.cvtColor(img, cv2.COLOR_BGR2LAB)
    l, a, b = cv2.split(lab)
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
    cl = clahe.apply(l)
    limg = cv2.merge((cl, a, b))
    return cv2.cvtColor(limg, cv2.COLOR_LAB2BGR)


def light_augment(img):
    h, w = img.shape[:2]

    if random.random() < 0.5:
        img = cv2.flip(img, 1)

    if random.random() < 0.4:
        angle = random.uniform(-6, 6)
        M = cv2.getRotationMatrix2D((w / 2, h / 2), angle, 1.0)
        img = cv2.warpAffine(img, M, (w, h), borderMode=cv2.BORDER_REPLICATE)

    if random.random() < 0.5:
        alpha = random.uniform(0.9, 1.15)
        beta = random.randint(-12, 12)
        img = cv2.convertScaleAbs(img, alpha=alpha, beta=beta)

    if random.random() < 0.9:
        img = apply_clahe(img)

    return img


def read_label_classes(label_path: Path):
    cls_set = set()
    with open(label_path, "r") as f:
        for line in f:
            if not line.strip():
                continue
            cls_id = int(line.split()[0])
            cls_set.add(cls_id)
    return cls_set


def count_current():
    counts = defaultdict(int)
    for lbl_file in LBL_DIR.glob("*.txt"):
        with open(lbl_file, "r") as f:
            for line in f:
                if not line.strip():
                    continue
                cid = int(line.split()[0])
                counts[cid] += 1
    return counts


def main():
    counts = count_current()
    print("Current counts:")
    for cid in range(10):
        print(cid, CLASS_NAMES[cid], counts.get(cid, 0))

    # collect image-label-class triples
    images_info = []
    for img_path in IMG_DIR.glob("*.*"):
        if img_path.suffix.lower() not in [".jpg", ".jpeg", ".png"]:
            continue
        lbl_path = LBL_DIR / (img_path.stem + ".txt")
        if not lbl_path.exists():
            continue
        cls_set = read_label_classes(lbl_path)
        images_info.append((img_path, lbl_path, cls_set))

    new_images = 0
    made_progress = True

    while made_progress:
        made_progress = False

        # stop condition
        if all(counts[c] >= TARGET_PER_CLASS for c in range(10)):
            break

        for img_path, lbl_path, cls_set in images_info:
            # find which classes in this image still need to be boosted
            need_boost = [c for c in cls_set if counts[c] < TARGET_PER_CLASS]
            if not need_boost:
                continue

            img = cv2.imread(str(img_path))
            if img is None:
                continue

            for _ in range(AUG_TRIES_PER_IMAGE):
                # re-check inside loop
                need_boost = [c for c in cls_set if counts[c] < TARGET_PER_CLASS]
                if not need_boost:
                    break

                aug = light_augment(img.copy())
                new_name = f"{img_path.stem}_bal{new_images}.jpg"
                new_img_path = IMG_DIR / new_name
                new_lbl_path = LBL_DIR / f"{img_path.stem}_bal{new_images}.txt"

                cv2.imwrite(str(new_img_path), aug)
                # copy label
                with open(lbl_path, "r") as fsrc, open(new_lbl_path, "w") as fdst:
                    fdst.write(fsrc.read())

                # IMPORTANT: only increment counts for classes that still needed boost
                for c in need_boost:
                    counts[c] += 1

                new_images += 1
                made_progress = True

            if all(counts[c] >= TARGET_PER_CLASS for c in range(10)):
                break

    print(f"\nDone. Created {new_images} images.")
    print("Final counts:")
    for cid in range(10):
        print(cid, CLASS_NAMES[cid], counts.get(cid, 0))


if __name__ == "__main__":
    main()


Current counts:
0 Comminuted 168
1 Greenstick 81
2 Healthy 54
3 Linear 21
4 Oblique Displaced 342
5 Oblique 48
6 Segmental 18
7 Spiral 66
8 Transverse Displaced 630
9 Transverse 120

Done. Created 4866 images.
Final counts:
0 Comminuted 650
1 Greenstick 650
2 Healthy 650
3 Linear 650
4 Oblique Displaced 650
5 Oblique 650
6 Segmental 650
7 Spiral 650
8 Transverse Displaced 650
9 Transverse 650


In [20]:
import os
from pathlib import Path
from collections import defaultdict
import shutil

DATA_ROOT = Path(r"C:\Final project 2\backend\datasets\train")
IMG_DIR = DATA_ROOT / "images"
LBL_DIR = DATA_ROOT / "labels"

TARGET = 650  # cap per class
REMOVE_DIR_IMG = IMG_DIR / "_removed"
REMOVE_DIR_LBL = LBL_DIR / "_removed"
REMOVE_DIR_IMG.mkdir(exist_ok=True)
REMOVE_DIR_LBL.mkdir(exist_ok=True)

# count how many we have per class
counts = defaultdict(int)

# we’ll iterate over ALL label files
label_files = sorted(LBL_DIR.glob("*.txt"))

for lbl_path in label_files:
    # read classes in this label
    with open(lbl_path, "r") as f:
        lines = [l.strip() for l in f.readlines() if l.strip()]
    if not lines:
        continue

    # this file may have multiple classes in it
    classes_in_file = set(int(l.split()[0]) for l in lines)

    # check if ANY of the classes in this file is already over target
    # if a file contains a class over target, we move the whole image+label out
    over = False
    for c in classes_in_file:
        if counts[c] >= TARGET:
            over = True
            break

    if over:
        # move files to removed
        img_path = IMG_DIR / (lbl_path.stem + ".jpg")
        if not img_path.exists():
            img_path = IMG_DIR / (lbl_path.stem + ".png")
        if img_path.exists():
            shutil.move(str(img_path), str(REMOVE_DIR_IMG / img_path.name))
        shutil.move(str(lbl_path), str(REMOVE_DIR_LBL / lbl_path.name))
        continue

    # otherwise we keep it and increment counts for the classes in it
    for c in classes_in_file:
        counts[c] += 1

print("Final kept counts:")
for c in range(10):
    print(c, counts[c])


Final kept counts:
0 650
1 650
2 650
3 650
4 650
5 650
6 650
7 650
8 650
9 604


In [21]:
from pathlib import Path

IMG_DIR = Path(r"C:\Final project 2\backend\datasets\train\images")
LBL_DIR = Path(r"C:\Final project 2\backend\datasets\train\labels")

imgs = {p.stem for p in IMG_DIR.iterdir() if p.suffix.lower() in [".jpg", ".jpeg", ".png"]}
lbls = {p.stem for p in LBL_DIR.iterdir() if p.suffix.lower() == ".txt"}

missing = imgs - lbls
print("Images without labels:", len(missing))
for m in list(missing)[:30]:
    print(m)


Images without labels: 0


In [15]:
import cv2, os
from pathlib import Path

IMG_DIR = Path(r"C:\Final project 2\backend\datasets\train\images")

for img_path in IMG_DIR.iterdir():
    if img_path.suffix.lower() not in [".jpg",".jpeg",".png"]:
        continue
    img = cv2.imread(str(img_path))
    if img is None:
        continue

    # CLAHE
    lab = cv2.cvtColor(img, cv2.COLOR_BGR2LAB)
    l,a,b = cv2.split(lab)
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
    cl = clahe.apply(l)
    limg = cv2.merge((cl,a,b))
    final = cv2.cvtColor(limg, cv2.COLOR_LAB2BGR)

    cv2.imwrite(str(img_path), final)  # overwrite


In [1]:
from ultralytics import YOLO
from pathlib import Path
import cv2
import os

# ---------------- CONFIG ----------------
# your trained model (the 10-class one)
MODEL_PATH = r"C:\Final project 2\backend\runs\yolo8s-fracture\weights\best.pt"

# FractAtlas root
DATA_ROOT = Path(r"C:\Final project 2\backend\datasets\FracAtlas")

# choose which split to test: "test" or "val" or "train"
SPLIT = "test"  # change to "val" if you want

IMG_DIR = DATA_ROOT / "images" / SPLIT
LBL_DIR = DATA_ROOT / "labels" / SPLIT  # typical YOLO structure; adjust if yours is different

CONF_THRES = 0.25  # detection threshold to decide "fractured"
IOU_THRES = 0.5    # not super important here since we're doing binary
# ----------------------------------------


def is_fractured_gt(img_path: Path) -> bool:
    """
    Ground truth: if there is a matching .txt in labels/split, we consider it FRACTURED.
    Else: HEALTHY.
    """
    lbl_path = LBL_DIR / (img_path.stem + ".txt")
    return lbl_path.exists() and lbl_path.stat().st_size > 0


def main():
    model = YOLO(MODEL_PATH)

    img_files = [p for p in IMG_DIR.iterdir() if p.suffix.lower() in [".jpg", ".jpeg", ".png"]]
    img_files.sort()

    TP = FP = TN = FN = 0

    for img_path in img_files:
        gt_fractured = is_fractured_gt(img_path)

        # run model
        results = model.predict(
            source=str(img_path),
            conf=CONF_THRES,
            imgsz=640,
            verbose=False
        )
        r = results[0]

        # your model has 10 classes (Comminuted, Greenstick, ... , Transverse)
        # but for FractAtlas we only care: did it detect ANYTHING?
        pred_fractured = len(r.boxes) > 0

        # update confusion matrix
        if gt_fractured and pred_fractured:
            TP += 1
        elif not gt_fractured and not pred_fractured:
            TN += 1
        elif not gt_fractured and pred_fractured:
            FP += 1
        elif gt_fractured and not pred_fractured:
            FN += 1

    total = TP + TN + FP + FN
    acc = (TP + TN) / total if total else 0
    precision = TP / (TP + FP) if (TP + FP) else 0
    recall = TP / (TP + FN) if (TP + FN) else 0
    f1 = 2 * precision * recall / (precision + recall) if (precision + recall) else 0

    print("=== FractAtlas binary evaluation ===")
    print(f"Images tested: {total}")
    print(f"TP (fracture correctly detected): {TP}")
    print(f"TN (healthy correctly ignored):   {TN}")
    print(f"FP (said fracture but healthy):   {FP}")
    print(f"FN (missed fracture):             {FN}")
    print()
    print(f"Accuracy:  {acc:.4f}")
    print(f"Precision: {precision:.4f}")
    print(f"Recall:    {recall:.4f}")
    print(f"F1-score:  {f1:.4f}")


if __name__ == "__main__":
    main()


WARNING Image Read Error C:\Final project 2\backend\datasets\FracAtlas\images\test\IMG0004084.jpg


ValueError: need at least one array to stack

In [17]:
from ultralytics import YOLO
from pathlib import Path
import cv2
from PIL import Image
import numpy as np

# ---------------- CONFIG ----------------
MODEL_PATH = r"C:\Final project 2\backend\runs\yolo8s-fracture\weights\best.pt"

DATA_ROOT = Path(r"C:\Final project 2\backend\datasets\FracAtlas")
SPLIT = "test"  # or "val"
IMG_DIR = DATA_ROOT / "images" / SPLIT
LBL_DIR = DATA_ROOT / "labels" / SPLIT

CONF_THRES = 0.15  # lowered to try to catch more fractures

# in your model: 2 = Healthy
HEALTHY_CLASS_ID = 2
# ----------------------------------------


def is_fractured_gt(img_path: Path) -> bool:
    lbl_path = LBL_DIR / (img_path.stem + ".txt")
    return lbl_path.exists() and lbl_path.stat().st_size > 0


def load_image_strong(path: Path):
    """Try several ways to read tricky JPEGs."""
    # 1) normal OpenCV
    img = cv2.imread(str(path))
    if img is not None:
        return img

    # 2) PIL
    try:
        pil_img = Image.open(path).convert("RGB")
        return cv2.cvtColor(np.array(pil_img), cv2.COLOR_RGB2BGR)
    except Exception:
        pass

    # 3) Windows-friendly imdecode
    try:
        data = np.fromfile(str(path), dtype=np.uint8)
        img = cv2.imdecode(data, cv2.IMREAD_COLOR)
        if img is not None:
            return img
    except Exception:
        pass

    return None


def clahe_enhance(bgr_img):
    lab = cv2.cvtColor(bgr_img, cv2.COLOR_BGR2LAB)
    l, a, b = cv2.split(lab)
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
    cl = clahe.apply(l)
    limg = cv2.merge((cl, a, b))
    return cv2.cvtColor(limg, cv2.COLOR_LAB2BGR)


def main():
    model = YOLO(MODEL_PATH)

    img_files = [p for p in IMG_DIR.iterdir()
                 if p.suffix.lower() in [".jpg", ".jpeg", ".png", ".bmp", ".tif", ".tiff"]]
    img_files.sort()

    TP = FP = TN = FN = 0
    skipped = 0

    for img_path in img_files:
        gt_fractured = is_fractured_gt(img_path)

        img = load_image_strong(img_path)
        if img is None:
            print(f"[skip] cannot read image: {img_path.name}")
            skipped += 1
            continue

        # histogram enhancement
        img = clahe_enhance(img)

        results = model.predict(
            source=img,
            conf=CONF_THRES,
            imgsz=640,
            verbose=False
        )
        r = results[0]

        # any NON-healthy detection = fractured
        pred_fractured = False
        for b in r.boxes:
            cls_id = int(b.cls[0])
            if cls_id != HEALTHY_CLASS_ID:
                pred_fractured = True
                break

        # confusion matrix
        if gt_fractured and pred_fractured:
            TP += 1
        elif not gt_fractured and not pred_fractured:
            TN += 1
        elif not gt_fractured and pred_fractured:
            FP += 1
        elif gt_fractured and not pred_fractured:
            FN += 1

    total = TP + TN + FP + FN
    acc = (TP + TN) / total if total else 0
    precision = TP / (TP + FP) if (TP + FP) else 0
    recall = TP / (TP + FN) if (TP + FN) else 0
    f1 = 2 * precision * recall / (precision + recall) if (precision + recall) else 0

    print("=== FractAtlas binary evaluation (CLAHE, strong loader, conf=0.15) ===")
    print(f"Images tested: {total}")
    print(f"Skipped unreadable images: {skipped}")
    print(f"TP: {TP}")
    print(f"TN: {TN}")
    print(f"FP: {FP}")
    print(f"FN: {FN}")
    print()
    print(f"Accuracy:  {acc:.4f}")
    print(f"Precision: {precision:.4f}")
    print(f"Recall:    {recall:.4f}")
    print(f"F1-score:  {f1:.4f}")


if __name__ == "__main__":
    main()


=== FractAtlas binary evaluation (CLAHE, strong loader, conf=0.15) ===
Images tested: 775
Skipped unreadable images: 0
TP: 20
TN: 589
FP: 50
FN: 116

Accuracy:  0.7858
Precision: 0.2857
Recall:    0.1471
F1-score:  0.1942


In [11]:
from PIL import Image, ImageFile
from pathlib import Path

ImageFile.LOAD_TRUNCATED_IMAGES = True

SRC_DIR = Path(r"C:\Final project 2\backend\datasets\FracAtlas\images\test")
OUT_DIR = SRC_DIR / "_fixed"
OUT_DIR.mkdir(exist_ok=True)

bad_files = [
    "IMG0004084.jpg",
    "IMG0004092.jpg",
    "IMG0004109.jpg",
    "IMG0004122.jpg",
    "IMG0004123.jpg",
    "IMG0004129.jpg",
    "IMG0004154.jpg",
    "IMG0004189.jpg",
    "IMG0004227.jpg",
    "IMG0004252.jpg",
    "IMG0004255.jpg",
    "IMG0004256.jpg",
    "IMG0004286.jpg",
    "IMG0004290.jpg",
    "IMG0004304.jpg",
    "IMG0004308.jpg",
]

for name in bad_files:
    src = SRC_DIR / name
    if not src.exists():
        print("[missing]", name)
        continue
    try:
        img = Image.open(src)
        img = img.convert("RGB")
        img.save(OUT_DIR / name, "JPEG", quality=95)
        print("[fixed]", name)
    except Exception as e:
        print("[still cannot read]", name, e)


[fixed] IMG0004084.jpg
[fixed] IMG0004092.jpg
[fixed] IMG0004109.jpg
[fixed] IMG0004122.jpg
[fixed] IMG0004123.jpg
[fixed] IMG0004129.jpg
[fixed] IMG0004154.jpg
[fixed] IMG0004189.jpg
[fixed] IMG0004227.jpg
[fixed] IMG0004252.jpg
[fixed] IMG0004255.jpg
[fixed] IMG0004256.jpg
[fixed] IMG0004286.jpg
[fixed] IMG0004290.jpg
[fixed] IMG0004304.jpg
[fixed] IMG0004308.jpg
